### Preparação

In [1]:
!pip install pandas
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.1/1.1 MB 13.3 MB/s eta 0:00:00
     ---------------------------------------- 4.8/4.8 MB 25.9 MB/s eta 0:00:00
     --------------------------------------- 22.0/22.0 MB 29.7 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 28.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [3]:
if not os.path.exists('data'):
  os.makedirs('data')

In [8]:
if not os.path.exists('data/caixas.csv'):
  !wget https://raw.githubusercontent.com/MBA-IA-GROUP-A/geoloc/master/data/caixas.csv -O data/caixas.csv

--2023-03-20 22:32:34--  https://raw.githubusercontent.com/MBA-IA-GROUP-A/geoloc/master/data/caixas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64593 (63K) [text/plain]
Saving to: ‘data/caixas.csv’

data/caixas.csv     100%[===================>]  63.08K  --.-KB/s    in 0.004s  

2023-03-20 22:32:34 (15.2 MB/s) - ‘data/caixas.csv’ saved [64593/64593]



In [9]:
if not os.path.exists('data/clientes.csv'):
  !wget https://raw.githubusercontent.com/MBA-IA-GROUP-A/geoloc/master/data/clientes.csv -O data/clientes.csv

--2023-03-20 22:32:36--  https://raw.githubusercontent.com/MBA-IA-GROUP-A/geoloc/master/data/clientes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 437034 (427K) [text/plain]
Saving to: ‘data/clientes.csv’

data/clientes.csv   100%[===================>] 426.79K  --.-KB/s    in 0.02s   

2023-03-20 22:32:36 (23.7 MB/s) - ‘data/clientes.csv’ saved [437034/437034]



In [10]:
caixas_df = pd.read_csv('./data/caixas.csv')
caixas_df.head()

,ID,latitude,longitude
0,0,-22.481797,-45.598899
1,1,-22.561752,-45.653728
2,2,-23.199113,-47.064822
3,3,-23.703180,-46.827509
4,4,-22.518834,-45.648207


In [11]:
clientes_df = pd.read_csv('data/clientes.csv')
clientes_df.head()

,ID,latitude,longitude
0,0,-22.692173,-45.559767
1,1,-23.434914,-46.638857
2,2,-23.593379,-46.444647
3,3,-22.479553,-45.678543
4,4,-23.811076,-46.645090


In [12]:
geo_caixas = gpd.GeoDataFrame(caixas_df, geometry=gpd.points_from_xy(caixas_df.longitude, caixas_df.latitude))
geo_caixas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         1500 non-null   int64   
 1   latitude   1500 non-null   float64 
 2   longitude  1500 non-null   float64 
 3   geometry   1500 non-null   geometry
dtypes: float64(2), geometry(1), int64(1)
memory usage: 47.0 KB


In [13]:
geo_clientes = gpd.GeoDataFrame(clientes_df, geometry=gpd.points_from_xy(clientes_df.longitude, clientes_df.latitude))
geo_clientes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         10000 non-null  int64   
 1   latitude   10000 non-null  float64 
 2   longitude  10000 non-null  float64 
 3   geometry   10000 non-null  geometry
dtypes: float64(2), geometry(1), int64(1)
memory usage: 312.6 KB


### Calculando a distancia até os caixas

In [14]:
geo_clientes['distancia_caixa'] = None

In [15]:
for index_caixa, caixa in geo_caixas.iterrows():
    # calcular a distância entre a caixa e cada cliente
    geo_clientes['distancia_caixa'] = geo_clientes.distance(caixa.geometry)
    # selecionar o cliente mais próximo
    cliente_proximo = geo_clientes.loc[geo_clientes['distancia_caixa'].idxmin()]
    # armazenar a distância para o cliente mais próximo
    geo_clientes.loc[cliente_proximo.name, 'distancia_caixa'] = cliente_proximo['distancia_caixa']

### Qual é o cliente que possui o caixa mais próximo?

In [16]:
cliente_mais_proximo = geo_clientes.loc[geo_clientes['distancia_caixa'].idxmin()]
f"O cliente com a menor distância em relação a qualquer caixa é o cliente {cliente_mais_proximo.ID}, há distancia de {round(cliente_mais_proximo.distancia_caixa, 3)}km"

'O cliente com a menor distância em relação a qualquer caixa é o cliente 5725, há distancia de 0.002km'

### Qual é o cliente que possui o caixa mais distante?

In [17]:
client_mais_distante = geo_clientes.loc[geo_clientes['distancia_caixa'].idxmax()]
f"O cliente com a maior distância em relação a qualquer caixa é o cliente {client_mais_distante.ID}, há distancia de {round(client_mais_distante.distancia_caixa, 3)}km"

'O cliente com a maior distância em relação a qualquer caixa é o cliente 4400, há distancia de 1.765km'

### Qual é a distância média das residências dos clientes em relação ao caixa mais próximo?

In [18]:
media_distancias = geo_clientes['distancia_caixa'].mean()
f"A média de distância entre os clientes e as caixas é de {round(media_distancias, 3)}km"

'A média de distância entre os clientes e as caixas é de 0.929km'